In [16]:
import numpy as np
import math

# Float to Spike converter
## FP Input Neuron

### Version 1

In [2]:
alpha = 17


def fp_d2s(x, K = 8, elim = 4, w = 1, alpha = 20):


    man_start = alpha
    exponent = 0

    while man_start > x and exponent < 2**elim - 1:
        exponent += 1
        man_start = man_start / 2


    print("man_start:\t", man_start)   
    print("exponent:\t", exponent)   
    print("exp elim:\t", 2**elim )

    T = man_start * 2* 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)

    print("T: \t\t", T)
    print("x:\t\t", x)
    print("α:\t\t", alpha)

    exponent = format(abs(exponent), "b")

    while len(exponent) < elim:
        exponent = "0" + exponent
    
    # mantessa
    z = []
    for t in T:
        if t <= x:
            x = x - t
            z.append("1")   
        else:
            z.append("0")


    man = ''.join(z)

    return exponent + man


print("output:\t\t", fp_d2s(x =  40, K = 4, elim = 4))

man_start:	 20
exponent:	 0
exp elim:	 16
T: 		 [20.  10.   5.   2.5]
x:		 40
α:		 20
output:		 00001111


### version 2

In [32]:
alpha = 17


def fp_d2s(x, K = 8, elim = 4, w = 1, alpha = 20):


    man_start = alpha
    exponent = 0

    print("test1:\t\t", x / alpha)
    print("test2:\t\t", 1 / (x / alpha))
    print("test3:\t\t", math.log10(5))


    while man_start > x and exponent < 2**elim - 1:
        exponent += 1
        man_start = man_start / 2


    print("man_start:\t", man_start)   
    print("exponent:\t", exponent)   
    print("exp elim:\t", 2**elim )

    T = man_start * 2* 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)

    print("T: \t\t", T)
    print("x:\t\t", x)
    print("α:\t\t", alpha)

    exponent = format(abs(exponent), "b")

    while len(exponent) < elim:
        exponent = "0" + exponent
    
    # mantessa
    z = []
    for t in T:
        if t <= x:
            x = x - t
            z.append("1")   
        else:
            z.append("0")


    man = ''.join(z)

    return exponent + man


print("output:\t\t", fp_d2s(x =  5, K = 4, elim = 4, alpha = 20))

test1:		 0.25
test2:		 4.0
test3:		 0.6989700043360189
man_start:	 5.0
exponent:	 2
exp elim:	 16
T: 		 [5.    2.5   1.25  0.625]
x:		 5
α:		 20
output:		 00101000


In [47]:
x = 2.5
alpha = 20
exponent = 0
elim = 4
man_start = alpha

while man_start > x and exponent < 2**elim - 1:
    exponent += 1
    man_start = man_start / 2

  
print("exponent:\t", exponent) 

print("test1:\t\t", x / alpha)
print("test2:\t\t", 1 / (x / alpha))
print("test3:\t\t", math.log(alpha, x / alpha))

exponent:	 3
test1:		 0.125
test2:		 8.0
test3:		 -1.4406426982957876


# Spike to Decimal Converter

### version 2

In [3]:
def fp_s2d(x, K = 8, elim = 4, w = 1, alpha = 20):
    exp = int(x[:elim], 2)
    man = list(x[elim:])

    man_start = alpha

    print("exp :", exp)
    print("man :", man)

    while exp > 0:
        exp -= 1
        man_start = man_start / 2
        
    print("man_start: ", man_start)   
    
    T = man_start * 2* 2**(-K) * np.array([float(2 ** (K - i)) for i in range(1, K + 1)]).astype(np.float32)

    print(T)
    # mantessa
    z = 0

    for i in range(len(man)):
        if man[i] == "1":
            z += T[i]

    return z

print(fp_s2d("00001111", K = 4, elim = 4))

exp : 0
man : ['1', '1', '1', '1']
man_start:  20
[20.  10.   5.   2.5]
37.5


# Current Alternative

main drawback is it's limit on being unable to convert input values which are either 0 or alpha-1

In [ ]:
import bitarray.util as bu
import math
def encode(x, alpha, exp_bits, mantissa_bits):
    # Calculate maximum mantissa value
    max_exp = (2 ** exp_bits) - 1
    print(max_exp)
    # Calculate what integer equivalent of mantissa of one will be
    mantissa_one = 2 ** mantissa_bits  #TODO function POW(2'.. check
    print(mantissa_one)
    # Get fraction of alpha we need to encode
    relative_x = x / alpha
    print(relative_x)
    assert relative_x < 1.0
    print(relative_x)
    # Calculate integer exponent (will be < 1) and clamp
    if x != 0:
        exponent = math.ceil(math.log2(relative_x)) #TODO look into log2 and ceil
        print(exponent)
        exponent = max(-max_exp, min(0, exponent)) #TODO math
        print(exponent)
    #exponent = 0 if x == 0: else exponent = 19
    # Calculate value this exponent will represent
    exponent_value = 2 ** exponent
    print("exponent value:", exponent_value)


    # Calculate floating point mantissa value
    mantissa = relative_x / exponent_value
    print(mantissa)
    assert mantissa < 1.0
    # Convert mantissa to fixed point bitarray and (negated) exponent to bitarray
    return (bu.int2ba(round(mantissa * mantissa_one), length=mantissa_bits),
            bu.int2ba(-exponent, length=exp_bits))
            
def decode(value, alpha):
    # Calculate what integer equivalent of mantissa of one will be
    mantissa_one = 2.0 ** len(value[0])
    # Extract components
    exp = bu.ba2int(value[1])
    mantissa = bu.ba2int(value[0])
    # Re-assemble floating point value
    return alpha * (2.0 ** -exp) * (mantissa / mantissa_one)
alpha = 20.0
code = encode(19, alpha, 4, 4)
print(code)
print(decode(code, alpha))